In [ ]:
import oracledb
from google.cloud.bigquery import Client
import os, json
from google.cloud import secretmanager
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DBT_ORCL_USER'),
    password=os.getenv('DBT_ORCL_PASS'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    project_id = os.getenv('BS_PROJECT_ID'), #byttet verdi
    table_uri = os.getenv('TS_v2_TABLE_URI'),  
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data_from_table(max_endret_tid):
    PROJECT_ID = oracle_secrets['project_id'] 
    TABLE_URI = oracle_secrets['table_uri'] 

    client = Client(project=PROJECT_ID)
    job = None
    if max_endret_tid is None:
        job = client.query(f"SELECT * FROM `{TABLE_URI}`")
    else:
        #max_opprettet_dato = pd.to_datetime(max_opprettet_dato, utc=True)
        job = client.query(f"SELECT * FROM `{TABLE_URI}` WHERE endret_tid > '{max_endret_tid}'")
        
    df = job.to_dataframe()
    
    return df

In [ ]:
def convert_to_json(df):

    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: json.loads(x) 
                if isinstance(x, str) and x.strip().startswith(('{', '[')) 
                else x )

    # Konverter DataFrame til en liste av JSON-objekter
    json_objs = json.loads(df.to_json(orient='records'))

    # Liste som inneholder tuples av alle rader som skal settes inn i tabellen
    extracted_values = []

    for json_obj in json_objs:
        # Serialiser JSON-objektet på nytt
        json_str = json.dumps(json_obj, ensure_ascii=False)
        ekstern_behandling_id = json_obj.get('ekstern_behandling_id')
        opprettet_tid = str(pd.to_datetime(json_obj.get('opprettet_tid'), unit='ms'))
        endret_tid = str(pd.to_datetime(json_obj.get('endret_tid'), unit='ms'))

        # Legg til i listen
        extracted_values.append((json_str, ekstern_behandling_id, opprettet_tid, endret_tid))

    return extracted_values


In [ ]:
def send_data():
    user = oracle_secrets['user'] + '[dvh_fam_ef]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            sql = """select max(endret_tid) from fam_ts_meta_data_v2"""
            cursor.execute(sql)
            result = cursor.fetchall()
            max_endret_tid = result[0][0]

            df = get_data_from_table(max_endret_tid)
            
            extracted_values = convert_to_json(df)
            
            # Insert JSON strings and extracted values into the table
            for json_str, ekstern_behandling_id, opprettet_tid, endret_tid in extracted_values:
                melding = json_str
                cursor.execute('''
                    INSERT INTO fam_ts_meta_data_v2 (melding, ekstern_behandling_id, opprettet_tid, endret_tid)
                    VALUES (:melding, :ekstern_behandling_id, TO_TIMESTAMP(:opprettet_tid, 'YYYY-MM-DD HH24:MI:SS.FF'), TO_TIMESTAMP(:endret_tid, 'YYYY-MM-DD HH24:MI:SS.FF'))
                ''', {
                    'melding': melding,
                    'ekstern_behandling_id': ekstern_behandling_id,
                    'opprettet_tid': opprettet_tid,
                    'endret_tid': endret_tid
                })
            
            # Commit and close connection
            conn.commit()
            #conn.close()

In [ ]:
send_data()